# Predicting 30-Day Hospital Readmissions (UCI Diabetes)
Quickstart notebook: train a Random Forest and view metrics.


In [ ]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
RANDOM_STATE=42
PATH='../data/raw/diabetes_130_us_hospitals.csv'


In [ ]:
df = pd.read_csv(PATH).replace('?', np.nan)
df = df.drop(columns=[c for c in ['encounter_id','patient_nbr'] if c in df.columns])
y = df['readmitted'].apply(lambda x: 1 if x=='<30' else 0).astype(int)
X = df.drop(columns=['readmitted'])
cat_cols = [c for c in X.columns if X[c].dtype=='O']
num_cols = [c for c in X.columns if c not in cat_cols]
preprocess = ColumnTransformer([
    ('num', SimpleImputer(strategy='median'), num_cols),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                      ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat_cols)
])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)


In [ ]:
clf = Pipeline([('prep', preprocess), ('rf', RandomForestClassifier(n_estimators=300, max_depth=20,
                                            random_state=RANDOM_STATE, class_weight='balanced'))])
clf.fit(X_train, y_train)
proba = clf.predict_proba(X_test)[:,1]
pred = (proba>=0.5).astype(int)
print('Accuracy:', accuracy_score(y_test, pred))
print('ROC-AUC:', roc_auc_score(y_test, proba))
print(classification_report(y_test, pred))
ConfusionMatrixDisplay.from_predictions(y_test, pred); plt.title('Confusion Matrix'); plt.tight_layout()